# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import getpass
import json

# Import API key
#from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("../WeatherPy/cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=getpass.getpass())

········


In [4]:
#Heatmap of humidity
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]
max_temp = cities_df["Max Temp"]
wind_speed = cities_df["Wind Speed"]
cloudiness = cities_df["Cloudiness"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 300, point_radius=5)
fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrow_cities_df = cities_df.loc[(cities_df["Max Temp"]<86) & (cities_df["Max Temp"]>32)&(cities_df["Wind Speed"] < 10)\
                                 & (cities_df["Cloudiness"]==0).dropna()]
narrow_cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
11,11,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
30,30,gobabis,0,NaN,1558378567,24,-22.45,18.97,68.00,2.24
31,31,qingdao,0,CN,1558378760,54,36.09,120.35,59.00,2.24


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = narrow_cities_df[['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''
hotel_df.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Country,Lat,Lng,Hotel Name
3,zhuhai,CN,40.71,112.04,
7,opuwo,NaN,-18.06,13.84,
11,castro,CL,-42.48,-73.76,
30,gobabis,NaN,-22.45,18.97,
31,qingdao,CN,36.09,120.35,


In [7]:
params = {
    "types": "hotel",
    "keyword" : "hotels",
    "radius":5000,
    "key": getpass.getpass()
}

········


In [12]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotel = requests.get(base_url, params=params)
    hotel = hotel.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
3,zhuhai,CN,40.71,112.04,
7,opuwo,NaN,-18.06,13.84,Opuwo Country Lodge
11,castro,CL,-42.48,-73.76,Apart Hotel y Cabañas Nomades
30,gobabis,NaN,-22.45,18.97,Goba Lodge
31,qingdao,CN,36.09,120.35,The Westin Qingdao
37,varhaug,NO,58.61,5.65,Huset ved havet
57,kolyvan,RU,55.31,82.74,
70,ponta do sol,BR,-20.63,-46.00,Escarpas do Lago
84,emerald,AU,-23.53,148.16,Emerald Executive Apartments
88,cidreira,BR,-30.17,-50.22,Pond Country Club


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))